<center><img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px; margin-left: 10px; max-height: 250px;" /></center>

# 2.3 Data Access and Basic Processing

<img src="https://openeo.org/images/openeo_logo.png"
     alt="openEO logo"
     style="float: center; margin-right: 10px; max-height: 100px;"/>
     
## Aggregate Operators with openEO

### `aggregate_temporal_period`: temporal aggregation with predefined intervals

Start importing the necessary libraries and initialize a local connection for Client-Side Processing.

In [ ]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

Create the starting Sentinel-2 datacube:

In [ ]:
url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"

spatial_extent = {"west": 11.4, "east": 11.42, "south": 45.5, "north": 45.52}
temporal_extent = ["2020-01-01", "2020-12-31"]
bands = ["red","green","blue"]
properties = {"eo:cloud_cover": dict(lt=50)}

s2_cube = local_conn.load_stac(url=url,
   spatial_extent=spatial_extent,
   temporal_extent=temporal_extent,
   bands=bands,
   properties=properties
)
s2_cube.execute()

We might be interested in aggregating our data over periods like week, month, year etc., defining what operation to use to combine the data available in the chosen period.

Using `aggregate_temporal_period` we can achieve this easily:

In [ ]:
s2_monthly_min = s2_cube.aggregate_temporal_period(period="month",reducer="min")
s2_monthly_min

Check what happens to the datacube inspecting the resulting xArray object. Now the `time` dimension has 12 labels, one for each month.

In [ ]:
s2_monthly_min.execute()

### `aggregate_spatial`: spatial aggregation with geometries

Before performing the operation on the satellite data with openEO, we need to get a vector data to be used in our workflow.

We will use the city boundaries of Bolzano (Italy) to perform an aggregation of the pixels within the specified geometry.

The geometry will be loaded from a public geoJSON file containing the boundaries of all the italian municipalities.

In [ ]:
import geopandas as gpd

italian_municipalities = gpd.read_file("https://raw.githubusercontent.com/openpolis/geojson-italy/refs/heads/master/comuni.geojson")
italian_municipalities.head()

We select the row corresponding to Bolzano and plot it with a base layer:

In [ ]:
import contextily as cx

bz_geom = italian_municipalities.loc[italian_municipalities["name"] == "Bolzano/Bozen"]

bz_wm = bz_geom.to_crs(epsg=3857)
ax = bz_wm.plot(figsize=(10, 10), alpha=0.5, edgecolor="k")
cx.add_basemap(ax)

We can now build our workflow with openEO, starting to load some Sentinel-2 data from STAC as before:

In [ ]:
url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"

spatial_extent = {"west": 11.25, "east": 11.44, "south": 46.4, "north": 46.6}
temporal_extent = ["2020-01-01", "2020-12-31"]
bands = ["red"]
properties = {"eo:cloud_cover": dict(lt=15),
             "earthsearch:boa_offset_applied": dict(eq=True)}

s2_cube = local_conn.load_stac(url=url,
   spatial_extent=spatial_extent,
   temporal_extent=temporal_extent,
   bands=bands,
   properties=properties
)

Now we call the `aggregate_spatial` process and compute the average over the Sentinel-2 pixels inside the city.

**Running this cell may take up to 2 minutes**

In [ ]:
from joblib import parallel_backend

with parallel_backend("threading"):
    bolzano_mean_xr = s2_cube.aggregate_spatial(
        geometries=bz_geom.geometry.values[0].geoms[0],
        reducer="mean"
    ).execute()


Finally, we can plot the resulting time series of values for a sample band:

In [ ]:
bolzano_mean_xr[:,:,0].plot()